<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/week13_%E7%B6%B2%E8%B7%AF%E8%B3%87%E6%96%99%E8%87%AA%E5%8B%95%E7%88%AC%E5%8F%96%E8%88%87%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ 1. 從網路上抓取新聞（Web Scraping）

以 ptt 為例，使用 requests + BeautifulSoup 抓取新聞標題與內容。

✅ 2. 分析新聞內容（NLP 分析）

用 SnowNLP（中文）進行情緒分析。

✅ 3. 視覺化分析結果

用 pandas 做表格處理，plotly 視覺化結果。

In [1]:
!pip install snownlp plotly beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=87e63f5de434ad5389e7998d63e3e221ed016237f8b45752dc2923bfac8ee396
  Stored in directory: /root/.cache/pip/wheels/8a/0a/37/f15b8568f5463f1427466f701e9d3ba514035eb703f885efee
Successfully built snownlp


In [2]:
import requests
from bs4 import BeautifulSoup
from snownlp import SnowNLP
import pandas as pd
import plotly.express as px

# 模擬 header 與 cookies（解鎖 Gossiping）
HEADERS = {
    'User-Agent': 'Mozilla/5.0'
}
COOKIES = {'over18': '1'}

In [34]:
rep = requests.get("https://www.ptt.cc/bbs/Gossiping/index.html", headers=HEADERS, cookies=COOKIES)

In [35]:
soup = BeautifulSoup(rep.text, 'html.parser')

In [36]:
entries = soup.select('div.r-ent')

In [37]:
entries

[<div class="r-ent">
 <div class="nrec"></div>
 <div class="title">
 <a href="/bbs/Gossiping/M.1763691021.A.C8C.html">[問卦] 糕點到了？</a>
 </div>
 <div class="meta">
 <div class="author">cigaretteass</div>
 <div class="article-menu">
 <div class="trigger">⋯</div>
 <div class="dropdown">
 <div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E5%95%8F%E5%8D%A6%5D+%E7%B3%95%E9%BB%9E%E5%88%B0%E4%BA%86%EF%BC%9F">搜尋同標題文章</a></div>
 <div class="item"><a href="/bbs/Gossiping/search?q=author%3Acigaretteass">搜尋看板內 cigaretteass 的文章</a></div>
 </div>
 </div>
 <div class="date">11/21</div>
 <div class="mark"></div>
 </div>
 </div>,
 <div class="r-ent">
 <div class="nrec"></div>
 <div class="title">
 <a href="/bbs/Gossiping/M.1763691034.A.DCC.html">[問卦] 看到羊腸就下跪了？</a>
 </div>
 <div class="meta">
 <div class="author">Mopack22926</div>
 <div class="article-menu">
 <div class="trigger">⋯</div>
 <div class="dropdown">
 <div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E5%95%8F%E5%8D%A6

In [48]:
SnowNLP("我心情不好").sentiments

0.4957853871897451

In [38]:
def get_ptt_articles(board='Stock', max_pages=2):
    base_url = f'https://www.ptt.cc/bbs/{board}/index.html'
    articles = []

    for _ in range(max_pages):
        res = requests.get(base_url, headers=HEADERS, cookies=COOKIES)
        soup = BeautifulSoup(res.text, 'html.parser')
        entries = soup.select('div.r-ent')

        for entry in entries:
            title_tag = entry.select_one('div.title > a')
            if title_tag:
                title = title_tag.text.strip()
                link = 'https://www.ptt.cc' + title_tag['href']

                try:
                    article_res = requests.get(link, headers=HEADERS, cookies=COOKIES)
                    article_soup = BeautifulSoup(article_res.text, 'html.parser')
                    content = article_soup.select_one('#main-content').text.split('--')[0]
                    content = '\n'.join(line for line in content.split('\n') if not line.startswith('※'))
                    sentiment = SnowNLP(content).sentiments
                except Exception as e:
                    content = ""
                    sentiment = None

                articles.append({
                    '標題': title,
                    '連結': link,
                    '情緒分數': sentiment
                })

        # 下一頁連結
        paging = soup.select_one('div.btn-group-paging > a.btn.wide:nth-child(2)')
        if paging:
            base_url = 'https://www.ptt.cc' + paging['href']
        else:
            break

    return pd.DataFrame(articles)

In [39]:
# 抓 PTT Stock 看板 2 頁
df = get_ptt_articles('Gossiping', max_pages=2)

In [40]:
df

,標題,連結,情緒分數
0,[問卦] 這些垃圾遊覽車接龍闖紅燈無法無天？,https://www.ptt.cc/bbs/Gossiping/M.1763691609....,6.403322e-12
1,[問卦] 中國人日旅年增40.7%是口嫌體正直嗎？,https://www.ptt.cc/bbs/Gossiping/M.1763691678....,3.254990e-02
2,[問卦] 普發一萬要救國旅還是要幫日本？,https://www.ptt.cc/bbs/Gossiping/M.1763691683....,9.319892e-01
3,[問卦] 慟！本肥的XR只能更新到18.7上不了26？,https://www.ptt.cc/bbs/Gossiping/M.1763691750....,5.436809e-10
4,[問卦] 板上有人去過北朝鮮旅遊過嗎！,https://www.ptt.cc/bbs/Gossiping/M.1763691764....,2.799516e-09
5,[公告] 八卦板板規(2025.01.21),https://www.ptt.cc/bbs/Gossiping/M.1737398137....,0.000000e+00
6,Re: [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。,https://www.ptt.cc/bbs/Gossiping/M.1759065775....,0.000000e+00
7,[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！,https://www.ptt.cc/bbs/Gossiping/M.1761980569....,0.000000e+00
8,[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊,https://www.ptt.cc/bbs/Gossiping/M.1763681667....,0.000000e+00
9,[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊,https://www.ptt.cc/bbs/Gossiping/M.1763682672....,0.000000e+00


In [41]:
df = df.dropna(subset=['情緒分數'])

In [42]:
df.head()

,標題,連結,情緒分數
0,[問卦] 這些垃圾遊覽車接龍闖紅燈無法無天？,https://www.ptt.cc/bbs/Gossiping/M.1763691609....,6.403322e-12
1,[問卦] 中國人日旅年增40.7%是口嫌體正直嗎？,https://www.ptt.cc/bbs/Gossiping/M.1763691678....,3.254990e-02
2,[問卦] 普發一萬要救國旅還是要幫日本？,https://www.ptt.cc/bbs/Gossiping/M.1763691683....,9.319892e-01
3,[問卦] 慟！本肥的XR只能更新到18.7上不了26？,https://www.ptt.cc/bbs/Gossiping/M.1763691750....,5.436809e-10
4,[問卦] 板上有人去過北朝鮮旅遊過嗎！,https://www.ptt.cc/bbs/Gossiping/M.1763691764....,2.799516e-09


In [43]:
# 畫圖
fig = px.bar(
    df.sort_values('情緒分數'),
    x='情緒分數',
    y='標題',
    orientation='h',
    title='PTT 文章情緒分析',
    height=600
)
fig.update_layout(yaxis=dict(automargin=True))
fig.show()

In [49]:
import jieba
from collections import Counter
import jieba.analyse

In [52]:
# jieba 中添加自定義詞彙
custom_dict = [
    '國立臺灣師範大學',
    '科技應用',
    '人力資源發展',
    '學系'
]

In [53]:
for word in custom_dict:
    jieba.add_word(word)

In [56]:
words = [w for w in jieba.cut("國立臺灣師範大學科技應用與人力資源發展學系") if w.strip()]

In [57]:
words

['國立臺灣師範大學', '科技應用', '與', '人力資源發展', '學系']

In [55]:
# 停用詞
stop_words = set([
    '所以', '好', '因為', '；', '的', '是', '了', '欲', '也', '在', '和', '就', '不', '有',
    '以', '與', '、', '為', '這個', '而', '「', '」', '，', '。', '《', '》', '對', '於', '\n', ' '
])

In [60]:
# 把所有標題串在一起當成一段文字
text = ''.join(df['標題'].astype(str).tolist())

In [61]:
text

'[問卦] 這些垃圾遊覽車接龍闖紅燈無法無天？[問卦] 中國人日旅年增40.7%是口嫌體正直嗎？[問卦] 普發一萬要救國旅還是要幫日本？[問卦] 慟！本肥的XR只能更新到18.7上不了26？[問卦] 板上有人去過北朝鮮旅遊過嗎！[公告] 八卦板板規(2025.01.21)Re: [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊[問卦] 糕點到了？[問卦] 看到羊腸就下跪了？Re: [新聞] 【非洲豬瘟之後】專訪被祭旗的特約獸醫師[問卦] 金曲獎什麼時候變成笑話的?[問卦] 42歲的單身姐姐說她看過世界[問卦] 阿館吃山羌的謎團是不是甘迺迪案一樣？[新聞] 不忍了！國中校長深夜親赴警局告賓賓哥2罪\u3000中警證實：已完Re: [問卦] 館長抗中到親中的轉折點是什麼[問卦] 有日本京都大學校園很不氣派的卦?Re: [問卦] 抵制全聯...有用嗎?Re: [心得] 竹北高鐵通勤與育兒的絕望感[新聞] 快新聞/陳其邁震怒!高雄田寮竟也見垃圾山[問卦] 台股上下洗是不是人也快中風了[新聞] 跟進賴清德吃日料挺日！林佳龍請客暴點[問卦] 這十二年，你過得好嗎？Re: [問卦] 館長抗中到親中的轉折點是什麼[問卦] 這廈門海鮮大排檔在台灣pr 值算多少？[問卦] 司馬懿智力二線 靠活得久才贏？[問卦] 遠傳跟中華電信該怎麼選？[問卦] 已經20天沒進去全聯了 有人比我久嗎？[公告] 八卦板板規(2025.01.21)Re: [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊'

In [62]:
# 避免輸出有 \n 跟空格
text = text.replace("\n", "").strip()

In [63]:
text

'[問卦] 這些垃圾遊覽車接龍闖紅燈無法無天？[問卦] 中國人日旅年增40.7%是口嫌體正直嗎？[問卦] 普發一萬要救國旅還是要幫日本？[問卦] 慟！本肥的XR只能更新到18.7上不了26？[問卦] 板上有人去過北朝鮮旅遊過嗎！[公告] 八卦板板規(2025.01.21)Re: [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊[問卦] 糕點到了？[問卦] 看到羊腸就下跪了？Re: [新聞] 【非洲豬瘟之後】專訪被祭旗的特約獸醫師[問卦] 金曲獎什麼時候變成笑話的?[問卦] 42歲的單身姐姐說她看過世界[問卦] 阿館吃山羌的謎團是不是甘迺迪案一樣？[新聞] 不忍了！國中校長深夜親赴警局告賓賓哥2罪\u3000中警證實：已完Re: [問卦] 館長抗中到親中的轉折點是什麼[問卦] 有日本京都大學校園很不氣派的卦?Re: [問卦] 抵制全聯...有用嗎?Re: [心得] 竹北高鐵通勤與育兒的絕望感[新聞] 快新聞/陳其邁震怒!高雄田寮竟也見垃圾山[問卦] 台股上下洗是不是人也快中風了[新聞] 跟進賴清德吃日料挺日！林佳龍請客暴點[問卦] 這十二年，你過得好嗎？Re: [問卦] 館長抗中到親中的轉折點是什麼[問卦] 這廈門海鮮大排檔在台灣pr 值算多少？[問卦] 司馬懿智力二線 靠活得久才贏？[問卦] 遠傳跟中華電信該怎麼選？[問卦] 已經20天沒進去全聯了 有人比我久嗎？[公告] 八卦板板規(2025.01.21)Re: [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊'

In [64]:
text = text.replace("\u3000", "").strip()

In [65]:
text

'[問卦] 這些垃圾遊覽車接龍闖紅燈無法無天？[問卦] 中國人日旅年增40.7%是口嫌體正直嗎？[問卦] 普發一萬要救國旅還是要幫日本？[問卦] 慟！本肥的XR只能更新到18.7上不了26？[問卦] 板上有人去過北朝鮮旅遊過嗎！[公告] 八卦板板規(2025.01.21)Re: [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊[問卦] 糕點到了？[問卦] 看到羊腸就下跪了？Re: [新聞] 【非洲豬瘟之後】專訪被祭旗的特約獸醫師[問卦] 金曲獎什麼時候變成笑話的?[問卦] 42歲的單身姐姐說她看過世界[問卦] 阿館吃山羌的謎團是不是甘迺迪案一樣？[新聞] 不忍了！國中校長深夜親赴警局告賓賓哥2罪中警證實：已完Re: [問卦] 館長抗中到親中的轉折點是什麼[問卦] 有日本京都大學校園很不氣派的卦?Re: [問卦] 抵制全聯...有用嗎?Re: [心得] 竹北高鐵通勤與育兒的絕望感[新聞] 快新聞/陳其邁震怒!高雄田寮竟也見垃圾山[問卦] 台股上下洗是不是人也快中風了[新聞] 跟進賴清德吃日料挺日！林佳龍請客暴點[問卦] 這十二年，你過得好嗎？Re: [問卦] 館長抗中到親中的轉折點是什麼[問卦] 這廈門海鮮大排檔在台灣pr 值算多少？[問卦] 司馬懿智力二線 靠活得久才贏？[問卦] 遠傳跟中華電信該怎麼選？[問卦] 已經20天沒進去全聯了 有人比我久嗎？[公告] 八卦板板規(2025.01.21)Re: [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊'

In [66]:
text = text.replace("Re:", "").strip()

In [67]:
text

'[問卦] 這些垃圾遊覽車接龍闖紅燈無法無天？[問卦] 中國人日旅年增40.7%是口嫌體正直嗎？[問卦] 普發一萬要救國旅還是要幫日本？[問卦] 慟！本肥的XR只能更新到18.7上不了26？[問卦] 板上有人去過北朝鮮旅遊過嗎！[公告] 八卦板板規(2025.01.21) [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊[問卦] 糕點到了？[問卦] 看到羊腸就下跪了？ [新聞] 【非洲豬瘟之後】專訪被祭旗的特約獸醫師[問卦] 金曲獎什麼時候變成笑話的?[問卦] 42歲的單身姐姐說她看過世界[問卦] 阿館吃山羌的謎團是不是甘迺迪案一樣？[新聞] 不忍了！國中校長深夜親赴警局告賓賓哥2罪中警證實：已完 [問卦] 館長抗中到親中的轉折點是什麼[問卦] 有日本京都大學校園很不氣派的卦? [問卦] 抵制全聯...有用嗎? [心得] 竹北高鐵通勤與育兒的絕望感[新聞] 快新聞/陳其邁震怒!高雄田寮竟也見垃圾山[問卦] 台股上下洗是不是人也快中風了[新聞] 跟進賴清德吃日料挺日！林佳龍請客暴點[問卦] 這十二年，你過得好嗎？ [問卦] 館長抗中到親中的轉折點是什麼[問卦] 這廈門海鮮大排檔在台灣pr 值算多少？[問卦] 司馬懿智力二線 靠活得久才贏？[問卦] 遠傳跟中華電信該怎麼選？[問卦] 已經20天沒進去全聯了 有人比我久嗎？[公告] 八卦板板規(2025.01.21) [問卦] 請勿前往光復區，鏟子超人任務艱鉅，病例增。[公告] 【八卦美學獎】梗圖吐槽，共鳴豹紋笑翻！[公告] 11/20 八卦踢報十大頭條 11月協尋 置底閒聊[公告] 11/21 八卦踢報十大頭條 11月協尋 置底閒聊'

In [58]:
filtered_words = [w for w in words if w not in stop_words]

In [59]:
filtered_words

['國立臺灣師範大學', '科技應用', '人力資源發展', '學系']

In [70]:
# jieba 中添加自定義詞彙
custom_dict = [
    '美股',
    '鴻海',
    '無法無天',
    '年增',
    '國旅'
]

In [71]:
for word in custom_dict:
    jieba.add_word(word)

In [72]:
# 先斷詞，這裡才會得到 words（修正點）
words = [w for w in jieba.cut(text) if w.strip()]

In [73]:
words

['[',
 '問卦',
 ']',
 '這些',
 '垃圾',
 '遊覽車',
 '接龍',
 '闖紅燈',
 '無法無天',
 '？',
 '[',
 '問卦',
 ']',
 '中國人',
 '日',
 '旅',
 '年增',
 '40.7%',
 '是',
 '口',
 '嫌體',
 '正直',
 '嗎',
 '？',
 '[',
 '問卦',
 ']',
 '普發',
 '一萬要',
 '救',
 '國旅',
 '還是',
 '要',
 '幫',
 '日本',
 '？',
 '[',
 '問卦',
 ']',
 '慟',
 '！',
 '本肥',
 '的',
 'XR',
 '只能',
 '更新',
 '到',
 '18.7',
 '上',
 '不了',
 '26',
 '？',
 '[',
 '問卦',
 ']',
 '板上',
 '有人',
 '去過',
 '北朝',
 '鮮',
 '旅遊過',
 '嗎',
 '！',
 '[',
 '公告',
 ']',
 '八卦',
 '板板規',
 '(',
 '2025.01',
 '.',
 '21',
 ')',
 '[',
 '問卦',
 ']',
 '請',
 '勿',
 '前往',
 '光',
 '復',
 '區',
 '，',
 '鏟子',
 '超人',
 '任務',
 '艱鉅',
 '，',
 '病例',
 '增',
 '。',
 '[',
 '公告',
 ']',
 '【',
 '八卦',
 '美學獎',
 '】',
 '梗圖',
 '吐',
 '槽',
 '，',
 '共鳴豹',
 '紋',
 '笑',
 '翻',
 '！',
 '[',
 '公告',
 ']',
 '11',
 '/',
 '20',
 '八卦',
 '踢',
 '報',
 '十大',
 '頭條',
 '11',
 '月',
 '協尋',
 '置底',
 '閒聊',
 '[',
 '公告',
 ']',
 '11',
 '/',
 '21',
 '八卦',
 '踢',
 '報',
 '十大',
 '頭條',
 '11',
 '月',
 '協尋',
 '置底',
 '閒聊',
 '[',
 '問卦',
 ']',
 '糕點',
 '到',
 '了',
 '？',
 '[',
 '問卦',
 ']',
 '看到',
 '羊腸',

In [74]:
# 過濾停用詞
filtered_words = [w for w in words if w not in stop_words]

In [75]:
filtered_words

['[',
 '問卦',
 ']',
 '這些',
 '垃圾',
 '遊覽車',
 '接龍',
 '闖紅燈',
 '無法無天',
 '？',
 '[',
 '問卦',
 ']',
 '中國人',
 '日',
 '旅',
 '年增',
 '40.7%',
 '口',
 '嫌體',
 '正直',
 '嗎',
 '？',
 '[',
 '問卦',
 ']',
 '普發',
 '一萬要',
 '救',
 '國旅',
 '還是',
 '要',
 '幫',
 '日本',
 '？',
 '[',
 '問卦',
 ']',
 '慟',
 '！',
 '本肥',
 'XR',
 '只能',
 '更新',
 '到',
 '18.7',
 '上',
 '不了',
 '26',
 '？',
 '[',
 '問卦',
 ']',
 '板上',
 '有人',
 '去過',
 '北朝',
 '鮮',
 '旅遊過',
 '嗎',
 '！',
 '[',
 '公告',
 ']',
 '八卦',
 '板板規',
 '(',
 '2025.01',
 '.',
 '21',
 ')',
 '[',
 '問卦',
 ']',
 '請',
 '勿',
 '前往',
 '光',
 '復',
 '區',
 '鏟子',
 '超人',
 '任務',
 '艱鉅',
 '病例',
 '增',
 '[',
 '公告',
 ']',
 '【',
 '八卦',
 '美學獎',
 '】',
 '梗圖',
 '吐',
 '槽',
 '共鳴豹',
 '紋',
 '笑',
 '翻',
 '！',
 '[',
 '公告',
 ']',
 '11',
 '/',
 '20',
 '八卦',
 '踢',
 '報',
 '十大',
 '頭條',
 '11',
 '月',
 '協尋',
 '置底',
 '閒聊',
 '[',
 '公告',
 ']',
 '11',
 '/',
 '21',
 '八卦',
 '踢',
 '報',
 '十大',
 '頭條',
 '11',
 '月',
 '協尋',
 '置底',
 '閒聊',
 '[',
 '問卦',
 ']',
 '糕點',
 '到',
 '？',
 '[',
 '問卦',
 ']',
 '看到',
 '羊腸',
 '下跪',
 '？',
 '[',
 '新聞',
 ']',
 '【',
 '非

In [76]:
# 統計詞頻
word_count = Counter(filtered_words)

In [77]:
# 只保留出現次數 > 1 的詞
filtered_word_count = {
    w: c for w, c in word_count.items() if c > 1
}

In [78]:
print("詞頻統計（出現次數 > 1）：")
for w, c in sorted(filtered_word_count.items(), key=lambda x: x[1], reverse=True):
    print(w, c)

詞頻統計（出現次數 > 1）：
[ 35
] 35
問卦 22
？ 12
公告 8
八卦 8
11 8
！ 6
嗎 5
/ 5
21 4
踢 4
報 4
十大 4
頭條 4
月 4
協尋 4
置底 4
閒聊 4
新聞 4
【 3
】 3
笑 3
20 3
? 3
垃圾 2
日 2
日本 2
到 2
有人 2
板板規 2
( 2
2025.01 2
. 2
) 2
請 2
勿 2
前往 2
光 2
復 2
區 2
鏟子 2
超人 2
任務 2
艱鉅 2
病例 2
增 2
美學獎 2
梗圖 2
吐 2
槽 2
共鳴豹 2
紋 2
翻 2
吃 2
山 2
是不是 2
館長 2
抗中到 2
親中 2
轉折點 2
什麼 2
全聯 2
跟 2
久 2


In [29]:
# 用 TF-IDF 抓關鍵字（修正 keywords 未定義）
keywords = jieba.analyse.extract_tags(text, topK=20)
print("\n關鍵字：", keywords)

# 這邊就直接把「每一則標題」當作一句話
sentences = df['標題'].astype(str).tolist()

# 依照「含有關鍵字的多寡」來排序句子，取前三句當簡易摘要
top_sentences = sorted(
    sentences,
    key=lambda s: sum(1 for w in jieba.cut(s) if w in keywords),
    reverse=True
)[:3]

summary = '。'.join(top_sentences).replace("\n", "")
print("\n簡易摘要：", summary)


關鍵字： ['Re', '請益', '新聞', '為何', 'nv', '財報', '還會', '亮眼', '現在', '美股', '閒聊', '心得', '越來', '2021', '細節', '大跌', 'AI', '做空', '鴻海', 'OpenAI']

簡易摘要： Re: [請益] 為何 nv 財報亮眼還會大跌阿。Re: [請益] 為何 nv 財報亮眼還會大跌阿。Re: [請益] 為何 nv 財報亮眼還會大跌阿
